## Download and prep the data

In [3]:
# Dataset
# In this homework, we will use the lead scoring dataset Bank Marketing dataset.

# run once
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv